## Bibliotecas a Utilizar

In [67]:
import numpy as np
import pandas as pd
from os import scandir

## Enlaces

## Trabajando los Data Frames - Función cleanData()
* Creando una columna con fecha para cada Data Frame
* Creando columna con variable de Riesgos de cada dato 
* Reemplazando los datos incorrectos
* Eliminando columnas que no sirven
* Posibilidad de guardar en archivo csv

## Función completa

In [68]:
def g_archivos(ruta = './data/Indicadores_Gestion/'):
    return sorted([archivo.path for archivo in scandir(ruta) if archivo.is_file()],reverse=True)


def enlace_df(l_enlaces):
    l_dataframes = []
    for enlace in l_enlaces:
        df = pd.read_excel(enlace, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
        l_dataframes.append(df)
    
    return l_dataframes


def tirar_resetear(df):
    """ Throw the useless rows and reset the index """
    df.drop(df.index[[0,1,2,3,4,5]],inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


def nuevas_columnas(df, fecha):
    """ Create and fill 'Fecha' column """
    fechas = ['06-2018', '03-2018',
        '12-2017', '09-2017', '06-2017', '03-2017',
        '12-2016', '09-2016', '06-2016', '03-2016',
        '12-2015', '09-2015', '06-2015', '03-2015',
        '12-2014', '09-2014', '06-2014', '03-2014',
        '12-2013', '09-2013', '06-2013', '03-2013',
        '12-2012', '09-2012', '06-2012', '03-2012',]
    
    df['Fecha'] = fechas[fecha]
    return df


def identificador_ramo(df):
    """ Fill 'Categoría' column with the apropiate insurance brunch """
    df['Categoría'] = None
    current_cat = 'Patrimoniales y Mixtas'
    
    for i in range(len(df)):
        if pd.isnull(df.iloc[i].NJ):
            current_cat = df.iloc[i]['Denominación']
        else:
            df.loc[i,'Categoría'] = current_cat
    
    current_cat = 'Patrimoniales y Mixtas'
    return df


def a_categoria(df):
    """ Convert 'Categoría' and 'Denominación' columns in category type, and eliminate the useless rows """
    df['Categoría'] = df['Categoría'].astype('category', inplace=True)
    df['Denominación'] = df['Denominación'].astype('category', inplace=True)
    df = df[~df.Categoría.isnull()]
    df.reset_index(drop=True, inplace=True)
    
    return df


def no_nan(df):
    """ Eliminate the wrong values """
    df.replace('//',0,inplace=True)
    df.replace('///',0,inplace=True)
    return df


def filtro_ramo(df, ramo):
    """ Filtered df by insurance brunch """
    ramos = {'PAT':'Patrimoniales y Mixtas', 
             'ART':'Riesgos del Trabajo', 
             'TRANSPORTE':'Transporte Público Pasajeros', 
             'VIDA':'Seguros de Vida', 
             'RETIRO':'Seguros de Retiro'}
    
    return df[ df['Categoría']== ramos[ramo]]


def guardado(df, ramo):
    """ Save in csv format the filtered info """
    df.to_csv('./clean_data/Indicadores_Gestion/{}.csv'.format(ramo))
    
def cleanData(ramo):

    l_enlaces = g_archivos()
    l_dataframes = enlace_df(l_enlaces)  # Lista de enlaces 
    largo = len(l_dataframes)            # Largo de la lista 
    df_Unido = pd.DataFrame()            # DF para guardar los df unidos
    

    for l, df in zip(range(largo),l_dataframes):
        tirar_resetear(df)               # U: Throw the useless rows and reset the index
        nuevas_columnas(df,l)            # U: Create and fill 'Fecha' column 
        identificador_ramo(df)           # U: Fill 'Categoría' column with the apropiate insurance brunch
        a_categoria(df)                  # U: Convert 'Categoría' and 'Denominación' columns in category type, and eliminate the useless rows
        no_nan(df)                       # U: Eliminate the wrong values
        df_Unido = df_Unido.append(filtro_ramo(df, ramo), ignore_index=True) # U: Append filtered df by insurance brunch
        
    guardado(df_Unido, ramo)             # U: Save in csv the filtered info
    return 


In [69]:
cleanData('PAT')

 **Posibles ramos a elegir en la función:**
* 'PAT' = Patrimoñales y Mixta
* 'ART' = Riesgos del Trabajo
* 'TRANSPORTE' = Transporte Público Pasajeros
* 'VIDA' = Seguros de Vida
* 'RETIRO' = Seguros de Retiro